In [1]:
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
path = 'DXYArea.csv'
df = pd.read_csv(path)
df['updateTime'] = pd.to_datetime(df['updateTime'], format='%Y-%m-%d %H:%M:%S')
df['year'] = df['updateTime'].dt.strftime('%Y')
df['month-day'] = df['updateTime'].dt.strftime('%m-%d')

In [2]:
df1 = df.loc[(df['provinceName'] != '澳门') & (df['provinceName'] != '台湾') & (df['provinceName'] != '香港') &  (df['provinceName'] != '中国') & (df['countryName'] == '中国') & (df['year'] == '2022')]
df1.drop(df1.columns[[0,1,2,3]], axis=1, inplace=True)
df1.drop(columns=['provinceEnglishName', 'province_zipCode', 'cityEnglishName', 'city_zipCode','cityName', 'city_confirmedCount', 'city_suspectedCount', 'city_curedCount', 'city_deadCount'], axis=1, inplace=True)

df1.reset_index(inplace=True, drop=True)
df2 = df1.groupby('provinceName').apply(lambda x: x.drop_duplicates('month-day', keep = 'first'))
df2.reset_index(drop=True, inplace=True)
df2['month'] = df2['updateTime'].dt.strftime('%m')


In [3]:
df2

,provinceName,province_confirmedCount,province_suspectedCount,province_curedCount,province_deadCount,updateTime,year,month-day,month
0,上海市,64978,393.0,64098,595,2022-12-14 09:39:05,2022,12-14,12
1,上海市,64958,393.0,64071,595,2022-12-13 16:08:50,2022,12-13,12
2,上海市,64936,393.0,64031,595,2022-12-12 16:05:33,2022,12-12,12
3,上海市,64918,393.0,64018,595,2022-12-11 16:06:04,2022,12-11,12
4,上海市,64910,393.0,63973,595,2022-12-10 16:58:53,2022,12-10,12
...,...,...,...,...,...,...,...,...,...
6103,黑龙江省,2064,394.0,2025,13,2022-02-10 10:08:45,2022,02-10,02
6104,黑龙江省,2064,394.0,2022,13,2022-02-08 16:57:25,2022,02-08,02
6105,黑龙江省,2064,394.0,2022,13,2022-02-06 19:16:20,2022,02-06,02
6106,黑龙江省,2064,394.0,2022,13,2022-02-03 19:25:02,2022,02-03,02


In [4]:
df3 = df2.groupby('provinceName').first()
df3['provinceName'] = df3.index
df3.reset_index(drop=True, inplace=True)
df3.drop(columns=['updateTime', 'year', 'month-day'], axis=1, inplace=True)
df3.columns = ['confirmedCount_end', 'suspectedCount_end', 'curedCount_end', 'deadCount_end', 'month','地区']
# get end of year data

In [5]:
df4 = df2.groupby('provinceName').last()
df4['provinceName'] = df4.index
df4.reset_index(drop=True, inplace=True)
df4.drop(columns=['updateTime', 'year', 'month-day'], axis=1, inplace=True)
df4.columns = ['confirmedCount_start', 'suspectedCount_start', 'curedCount_start', 'deadCount_start', 'month','地区']
# get start of year data

In [6]:
df5 = pd.merge(df4, df3,on='地区')
df5.drop(columns=['month_x','month_y'])

,confirmedCount_start,suspectedCount_start,curedCount_start,deadCount_start,地区,confirmedCount_end,suspectedCount_end,curedCount_end,deadCount_end
0,3103,393.0,2915,7,上海市,64978,393.0,64098,595
1,1817,22.0,1732,2,云南省,5007,23.0,4072,2
2,1186,35.0,1129,1,内蒙古自治区,8237,35.0,7962,1
3,1211,164.0,1190,9,北京市,25249,164.0,14283,13
4,589,19.0,582,3,吉林省,40481,19.0,40422,5
5,1321,22.0,1283,3,四川省,11315,22.0,9929,5
6,589,50.0,527,3,天津市,2627,50.0,2566,3
7,122,0.0,122,0,宁夏回族自治区,238,0.0,237,0
8,1009,2.0,1003,6,安徽省,1699,2.0,1654,6
9,1043,14.0,1012,7,山东省,4564,14.0,4131,8


In [7]:
df6 = df.loc[(df['countryName'] == '中国') & (df['year'] == '2022')]
df6.drop(df1.columns[[0,1,2,3]], axis=1, inplace=True)
df6.drop(columns=['provinceEnglishName', 'province_zipCode', 'cityEnglishName', 'city_zipCode','cityName', 'city_confirmedCount', 'city_suspectedCount', 'city_curedCount', 'city_deadCount'], axis=1, inplace=True)
df6.reset_index(inplace=True, drop=True)
df6 = df6.drop_duplicates('month-day', keep='first')
df6

,continentName,continentEnglishName,countryName,countryEnglishName,province_deadCount,updateTime,year,month-day
0,亚洲,Asia,中国,China,2,2022-12-14 17:35:24,2022,12-14
579,亚洲,Asia,中国,China,14722,2022-12-13 17:29:42,2022,12-13
1463,亚洲,Asia,中国,China,14710,2022-12-12 17:20:53,2022,12-12
2394,亚洲,Asia,中国,China,14679,2022-12-11 17:18:26,2022,12-11
3357,亚洲,Asia,中国,China,5,2022-12-10 18:01:34,2022,12-10
...,...,...,...,...,...,...,...,...
185756,亚洲,Asia,中国,China,1,2022-01-05 19:22:23,2022,01-05
186141,亚洲,Asia,中国,China,3,2022-01-04 17:35:10,2022,01-04
186490,亚洲,Asia,中国,China,850,2022-01-03 17:59:34,2022,01-03
186771,亚洲,Asia,中国,China,2,2022-01-02 19:24:35,2022,01-02


In [8]:
import numpy as np
df4 = df2.groupby(['provinceName','month']).last()
df4.drop(columns=['updateTime', 'year', 'month-day'], axis=1, inplace=True)
df4.columns = ['confirmedCount_start', 'suspectedCount_start', 'curedCount_start', 'deadCount_start']
slice1 = []
slice2 = []
for i,j in df4.index:
    slice1.append(i)
    slice2.append(j)

df4['地区'] = slice1
df4['month'] = slice2
df4.reset_index(drop=True,inplace=True)

In [9]:
df4

,confirmedCount_start,suspectedCount_start,curedCount_start,deadCount_start,地区,month
0,3103,393.0,2915,7,上海市,01
1,3792,393.0,3522,7,上海市,02
2,4351,393.0,3863,7,上海市,03
3,6454,393.0,4998,7,上海市,04
4,58341,393.0,37647,429,上海市,05
...,...,...,...,...,...,...
343,2993,430.0,2977,13,黑龙江省,08
344,3084,430.0,3019,13,黑龙江省,09
345,3389,430.0,3235,13,黑龙江省,10
346,3540,430.0,3461,13,黑龙江省,11


In [10]:
df5 = df2.groupby(['provinceName','month']).first()
df5.drop(columns=['updateTime', 'year', 'month-day'], axis=1, inplace=True)
df5.columns = ['confirmedCount_end', 'suspectedCount_end', 'curedCount_end', 'deadCount_end']
slice1 = []
slice2 = []
for i,j in df5.index:
    slice1.append(i)
    slice2.append(j)
df5['地区'] = slice1
df5['month'] = slice2
df5.reset_index(drop=True,inplace=True)

In [11]:
df6 = pd.merge(df4,df5)
df6['confirmedCount'] = df6['confirmedCount_end'] - df6['confirmedCount_start']
df6['suspectedCount'] = df6['suspectedCount_end'] - df6['suspectedCount_start']
df6['deadCount'] = df6['deadCount_end'] - df6['deadCount_start']
df6['curedCount'] = df6['curedCount_end'] - df6['curedCount_start']
df6.drop(columns=['confirmedCount_end', 'suspectedCount_end', 'curedCount_end', 'deadCount_end','confirmedCount_start', 'suspectedCount_start', 'curedCount_start', 'deadCount_start'], axis=1, inplace=True)
df6

,地区,month,confirmedCount,suspectedCount,deadCount,curedCount
0,上海市,01,586,0.0,0,387
1,上海市,02,518,0.0,0,329
2,上海市,03,1738,0.0,0,1109
3,上海市,04,51098,0.0,384,29593
4,上海市,05,4662,0.0,166,24029
...,...,...,...,...,...,...
343,黑龙江省,08,81,0.0,0,42
344,黑龙江省,09,300,0.0,0,213
345,黑龙江省,10,110,0.0,0,220
346,黑龙江省,11,894,0.0,0,154


In [12]:
count = df6.groupby('地区').size().reset_index(name='counts')
count = count.loc[count['counts']==12]
count.reset_index(drop=True)

,地区,counts
0,上海市,12
1,云南省,12
2,内蒙古自治区,12
3,北京市,12
4,吉林省,12
5,四川省,12
6,天津市,12
7,安徽省,12
8,山东省,12
9,山西省,12


In [71]:
lst = count['地区']
csv = df6.loc[df6['地区'].isin(lst)]
csv.to_csv('Covid_Data_Month.csv')
# for panel data analyze

In [68]:
csv

,地区,month,confirmedCount,suspectedCount,deadCount
0,上海市,01,586,0.0,0
1,上海市,02,518,0.0,0
2,上海市,03,1738,0.0,0
3,上海市,04,51098,0.0,384
4,上海市,05,4662,0.0,166
...,...,...,...,...,...
343,黑龙江省,08,81,0.0,0
344,黑龙江省,09,300,0.0,0
345,黑龙江省,10,110,0.0,0
346,黑龙江省,11,894,0.0,0


In [13]:
lst = list(set(df2['provinceName']))
len(lst)

31

In [14]:
df_month = df.loc[(df['provinceName'] != '澳门') & (df['provinceName'] != '台湾') & (df['provinceName'] != '香港') &  (df['provinceName'] != '中国') & (df['countryName'] == '中国') & (df['year'] != '2020')]
df_month['month'] = df_month['updateTime'].dt.strftime('%m')
df_month.drop(df_month.columns[[0,1,2,3]], axis=1, inplace=True)
df_month.drop(columns=['provinceEnglishName', 'province_zipCode', 'cityEnglishName', 'city_zipCode','cityName', 'city_confirmedCount', 'city_suspectedCount', 'city_curedCount', 'city_deadCount'], axis=1, inplace=True)

df_month = df_month.groupby('provinceName').apply(lambda x: x.drop_duplicates('month-day', keep = 'first'))
df_month.reset_index(inplace=True, drop=True)
df_month['year-month'] = df_month['updateTime'].dt.strftime('%Y-%m')
df_month

,provinceName,province_confirmedCount,province_suspectedCount,province_curedCount,province_deadCount,updateTime,year,month-day,month,year-month
0,上海市,64978,393.0,64098,595,2022-12-14 09:39:05,2022,12-14,12,2022-12
1,上海市,64958,393.0,64071,595,2022-12-13 16:08:50,2022,12-13,12,2022-12
2,上海市,64936,393.0,64031,595,2022-12-12 16:05:33,2022,12-12,12,2022-12
3,上海市,64918,393.0,64018,595,2022-12-11 16:06:04,2022,12-11,12,2022-12
4,上海市,64910,393.0,63973,595,2022-12-10 16:58:53,2022,12-10,12,2022-12
...,...,...,...,...,...,...,...,...,...,...
7733,黑龙江省,971,394.0,949,13,2021-01-07 16:16:25,2021,01-07,01,2021-01
7734,黑龙江省,970,394.0,949,13,2021-01-06 09:06:33,2021,01-06,01,2021-01
7735,黑龙江省,969,394.0,948,13,2021-01-05 10:20:58,2021,01-05,01,2021-01
7736,黑龙江省,969,394.0,946,13,2021-01-04 08:05:22,2021,01-04,01,2021-01


In [16]:
table = {
    'month':['01','02','03','04','05','06','07','08','09','10','11','12','01','02','03','04','05','06','07','08','09','10','11','12'],
    'month1':['01','02','03','04','05','06','07','08','09','10','11','12','01','02','03','04','05','06','07','08','09','10','11','12'],
    'year':['2021','2021','2021','2021','2021','2021','2021','2021','2021','2021','2021','2021','2022','2022','2022','2022','2022','2022','2022','2022','2022','2022','2022','2022'],
    'year1':['2021','2021','2021','2021','2021','2021','2021','2021','2021','2021','2021','2021','2022','2022','2022','2022','2022','2022','2022','2022','2022','2022','2022','2022'],
    'confirmedCount_month':[None]*24,
    'suspectedCount_month':[None]*24,
    'curedCount_month':[None]*24,
    'deadCount_month':[None]*24
}
table = pd.DataFrame(table)
table = table.groupby(['year','month']).last()

In [17]:
months = ['01','02','03','04','05','06','07','08','09','10','11','12','01','02','03','04','05','06','07','08','09','10','11','12']
years = ['2021','2022']
for year in years:

    for month in months:
        tmp_table = {
            'confirmedCount_month':[],
            'suspectedCount_month':[],
            'curedCount_month':[],
            'deadCount_month':[]
        }
        tmp_table = pd.DataFrame(tmp_table)

        for i in lst:
            ts = '%s-%s'%(year,month)
            tmp = df_month.loc[(df_month['year-month']==ts)&(df_month['provinceName']==i)]
            if(len(tmp)>=1):
                ccount = tmp.iloc[0,1] - tmp.iloc[-1,1]
                scount = tmp.iloc[0,2] - tmp.iloc[-1,2]
                crcount = tmp.iloc[0,3] - tmp.iloc[-1,3]
                dcount = tmp.iloc[0,4] - tmp.iloc[-1,4]
            tmp_table.loc[len(tmp_table)] = [ccount,scount,crcount,dcount]
            
        ccount = tmp_table['confirmedCount_month'].sum()
        scount = tmp_table['suspectedCount_month'].sum()
        crcount = tmp_table['curedCount_month'].sum()
        dcount = tmp_table['deadCount_month'].sum()
        table.loc[(table['month1']==month)&(table['year1']==year)] = [month, year, ccount,scount,crcount,dcount]

table

month1 year1 confirmedCount_month suspectedCount_month  \
year month                                                          
2021 01        01  2021               2110.0                 14.0   
     02        02  2021                118.0                  0.0   
     03        03  2021                605.0                  0.0   
     04        04  2021                257.0                  0.0   
     05        05  2021                664.0                  0.0   
     06        06  2021                845.0                 14.0   
     07        07  2021                476.0                 25.0   
     08        08  2021               5020.0                150.0   
     09        09  2021               5032.0                150.0   
     10        10  2021                783.0                  0.0   
     11        11  2021               1456.0                  0.0   
     12        12  2021               2436.0                  0.0   
2022 01        01  2022               3692.0                  0.0   
     02        02  2022               3920.0                  0.0   
     03        03  2022              40240.0                  0.0   
     04        04  2022              64631.0                 36.0   
     05        05  2022               7798.0                  1.0   
     06        06  2022               1888.0                  0.0   
     07        07  2022               4256.0                  0.0   
     08        08  2022              13478.0                  0.0   
     09        09  2022               7001.0                  0.0   
     10        10  2022               9335.0                  0.0   
     11        11  2022              56562.0                  0.0   
     12        12  2022              45866.0                  0.0   

           curedCount_month deadCount_month  
year month                                   
2021 01               853.0             2.0  
     02              1358.0             0.0  
     03              6798.0             0.0  
     04               209.0             0.0  
     05               627.0             0.0  
     06               733.0             0.0  
     07               116.0             0.0  
     08              1026.0             0.0  
     09              1520.0             0.0  
     10               677.0             0.0  
     11              1475.0             0.0  
     12              1033.0             0.0  
2022 01              3837.0             0.0  
     02              2191.0             0.0  
     03             13764.0             2.0  
     04             67891.0           384.0  
     05             29742.0           166.0  
     06              3627.0             0.0  
     07              2912.0             0.0  
     08              8815.0             0.0  
     09             10317.0             0.0  
     10              8045.0             0.0  
     11             26747.0             7.0  
     12             48138.0             2.0

In [174]:
table.to_csv('data/Covid_Data_Month_Total.csv',index=True)
#Monthly data